In [53]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import torch
from sklearn.model_selection import train_test_split
import glob
import os

In [54]:
# Path to the data folder
data_path = "Mod7_data/data"
file_pattern = os.path.join(data_path, "fof_subhalo_tab_000.*.hdf5") 
files = sorted(glob.glob(file_pattern))

# Loop through all files
for file in files:
    with h5py.File(file, 'r') as f:
        print(f"File: {file}")
        #print("keys:")
        #print(list(f.keys()))
        #print("-" * 40)

File: Mod7_data/data/fof_subhalo_tab_000.0.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.1.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.10.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.100.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.101.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.102.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.103.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.104.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.105.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.106.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.107.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.108.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.109.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.11.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.110.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.111.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.112.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.113.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.114.hdf5
File: Mod7_data/data/fof_subhalo_tab_000.115.hdf5
File: 

In [55]:
with h5py.File(files[0], 'r') as f: #files[0] = Mod7_data/data/fof_subhalo_tab_000.0.hdf5 (1st data file)
    data = f['Subhalo'].keys()   
    print(data)

<KeysViewHDF5 ['SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloCM', 'SubhaloFlag', 'SubhaloGasMetalFractions', 'SubhaloGasMetalFractionsHalfRad', 'SubhaloGasMetalFractionsMaxRad', 'SubhaloGasMetalFractionsSfr', 'SubhaloGasMetalFractionsSfrWeighted', 'SubhaloGasMetallicity', 'SubhaloGasMetallicityHalfRad', 'SubhaloGasMetallicityMaxRad', 'SubhaloGasMetallicitySfr', 'SubhaloGasMetallicitySfrWeighted', 'SubhaloGrNr', 'SubhaloHalfmassRad', 'SubhaloHalfmassRadType', 'SubhaloIDMostbound', 'SubhaloLen', 'SubhaloLenType', 'SubhaloMass', 'SubhaloMassInHalfRad', 'SubhaloMassInHalfRadType', 'SubhaloMassInMaxRad', 'SubhaloMassInMaxRadType', 'SubhaloMassInRad', 'SubhaloMassInRadType', 'SubhaloMassType', 'SubhaloParent', 'SubhaloPos', 'SubhaloSFR', 'SubhaloSFRinHalfRad', 'SubhaloSFRinMaxRad', 'SubhaloSFRinRad', 'SubhaloSpin', 'SubhaloStarMetalFractions', 'SubhaloStarMetalFractionsHalfRad', 'SubhaloStarMetalFractionsMaxRad', 'SubhaloStarMetallicity', 'SubhaloStarMetallicityHalfRad', 'SubhaloStarMetallicityM

## Subhalos are galaxy or galaxy-groups which can be considered as nodes for our GNN study. And the keys under subhalo directory can be used as different features for a specific node.

### Define feature matrix: considered only subhalo-BH mass and subhalo mass as features

In [56]:
all_bh_mass = []
all_mass = []

for file in files:
    with h5py.File(file, 'r') as f:
        bh_mass = f['Subhalo']['SubhaloBHMass'][:]
        mass = f['Subhalo']['SubhaloMass'][:]
        
        all_bh_mass.append(bh_mass)
        all_mass.append(mass)

# Concatenate across all files
SubhaloBHMass = np.concatenate(all_bh_mass)
SubhaloMass = np.concatenate(all_mass)

# Stack them into a feature matrix: shape (num_subhalos_total, 2)
Features = np.stack([SubhaloBHMass, SubhaloMass], axis=1)

### Define edges: So we need to know the location of all nodes (subhalos) and will use k-nearest neighbours to connect them

In [57]:
all_pos = []

for file in files:
    with h5py.File(file, 'r') as f:
        pos = f['Subhalo']['SubhaloPos'][:]  # shape: (N, 3)
        all_pos.append(pos)

SubhaloPos = np.concatenate(all_pos)  # shape: (total_N, 3)

In [58]:
from sklearn.neighbors import NearestNeighbors

k = 10  # Number of neighbors per node

knn = NearestNeighbors(n_neighbors=k+1)  # +1 because self is included
knn.fit(SubhaloPos)
distances, indices = knn.kneighbors(SubhaloPos) #indices is shape (N, k+1) where each row contains the indices of the nearest neighbors (including the node itself at index 0).

### Building edge index

In [59]:
# Skip self-edges: use indices[:, 1:]
src = np.repeat(np.arange(indices.shape[0]), k)
dst = indices[:, 1:].flatten()

edge_index = torch.tensor([src, dst], dtype=torch.long)

### Building PyTorch Geometric Data Object

### Node prediction (1 if BH mass is large relative to total mass)

In [85]:
from torch_geometric.data import Data

# Label: 1 if BH mass is large relative to total mass
labels = (x[:, 1] > 2.7).long()
#data.y = labels
x = torch.tensor(Features, dtype=torch.float) #node features = [num_nodes, num_features]
data = Data(x=x, edge_index=edge_index, y=labels) 
#data.y = labels

### Visualize the graph data (taking long time to generate the graphical input data)

In [52]:
import networkx as nx
# Convert to NetworkX graph
G = nx.Graph()
edge_list = data.edge_index.t().tolist()  # Convert edge_index to list of edges

# Add nodes with attributes (features)
for i, feature in enumerate(data.x):
    G.add_node(i, feature=feature.numpy())

# Add edges
G.add_edges_from(edge_list)

# Draw the graph
plt.figure(figsize=(8, 8))
pos = nx.spring_layout(G)  # Spring layout for better node spacing
nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', font_size=15, font_weight='bold', edge_color='gray')

# Show the plot
plt.show()

KeyboardInterrupt: 

<Figure size 800x800 with 0 Axes>

In [86]:
data.train_mask = torch.rand(data.num_nodes) < 0.8  # 80% train
data.test_mask = ~data.train_mask

In [87]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(2, 16)
        self.conv2 = GCNConv(16, 2)  # Assuming binary classification

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [88]:
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.NLLLoss()

for epoch in range(201):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)                # Forward pass
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Train loss
    loss.backward()
    optimizer.step()

    # Evaluate
    model.eval()
    _, pred = out.max(dim=1)  # Get predicted classes
    correct = pred[data.test_mask] == data.y[data.test_mask]
    acc = int(correct.sum()) / int(data.test_mask.sum())

    if epoch % 20 == 0:
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}, Test Acc: {acc:.4f}')

Epoch 000, Loss: 0.6931, Test Acc: 1.0000
Epoch 020, Loss: 0.2301, Test Acc: 1.0000
Epoch 040, Loss: 0.0431, Test Acc: 1.0000
Epoch 060, Loss: 0.0109, Test Acc: 1.0000
Epoch 080, Loss: 0.0045, Test Acc: 1.0000
Epoch 100, Loss: 0.0025, Test Acc: 1.0000
Epoch 120, Loss: 0.0008, Test Acc: 1.0000
Epoch 140, Loss: 0.0003, Test Acc: 1.0000
Epoch 160, Loss: 0.0004, Test Acc: 1.0000
Epoch 180, Loss: 0.0004, Test Acc: 1.0000
Epoch 200, Loss: 0.0004, Test Acc: 1.0000


In [89]:
model.eval()
final_output = model(data.x, data.edge_index)
predicted_labels = final_output.argmax(dim=1)

In [90]:
print(final_output)

tensor([[-1.2707e-04, -8.9712e+00],
        [-6.5444e-05, -9.6341e+00],
        [-3.6829e-04, -7.9068e+00],
        ...,
        [-2.1694e-04, -8.4358e+00],
        [-5.8026e-04, -7.4524e+00],
        [-5.7621e-04, -7.4593e+00]], grad_fn=<LogSoftmaxBackward0>)


In [91]:
print(predicted_labels)

tensor([0, 0, 0,  ..., 0, 0, 0])
